In [2]:
import warnings
warnings.simplefilter('ignore')

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import pandas as pd 
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

from lightgbm import LGBMClassifier
from imblearn.under_sampling import RandomUnderSampler
from lightgbm import early_stopping

In [ ]:
train = pd.read_csv('train_df.csv')
test = pd.read_csv('test_df.csv')

#check unique values later 
train['body_system_2'] = train['body_system_2'].replace('Undefined Diagnoses', 'Undefined diagnoses')
test['body_system_2'] = test['body_system_2'].replace('Undefined Diagnoses', 'Undefined diagnoses')